# Calculating the reserve Factors of the panels 

# Please enter your name 

In [1]:
name = "yannis"

# Imports 

In [2]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))
sys.path.insert(0, os.path.abspath('../optimization'))

from formulas.panels import *
from formulas.helpers import *
from optimization.panelBuckReverse import *
 

In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

# Constants (Changed based on your data)

In [4]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
nu = personal_data[2]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}, nu = {nu}. Please verify.")

#Panel constants 
length = 750
width = 200


Your personal data is: sigma_yield = 490, EModulus = 65241.07, nu = 0.34. Please verify.


# Open stress values & properties of panels 

In [5]:
# New version of importing panel stress file
paneldf = pd.read_csv(f'../data/{name}/panel.csv')


#Import panel properties and reformat
panelPropertiesdf = pd.read_csv(f'../data/{name}/panel_properties.csv', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.drop(columns=['Component Name', 'mass'])


paneldf.head(5)


,Element ID,Component Name,sigmaXX,sigmaYY,sigmaXY,Load Case
0,1,panel1,-65.783,12.580,25.898,Subcase 1 (LC1)
1,2,panel1,-81.868,20.594,26.738,Subcase 1 (LC1)
2,3,panel1,-66.730,20.466,36.481,Subcase 1 (LC1)
3,4,panel2,-96.378,2.383,34.062,Subcase 1 (LC1)
4,5,panel2,-88.315,25.010,26.685,Subcase 1 (LC1)


# Split up the loadcases 

In [6]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Component Name,sigmaXX,sigmaYY,sigmaXY,Load Case,thickness
Element ID,,,,,,
1,panel1,-65.783,12.580,25.898,Subcase 1 (LC1),4.0
2,panel1,-81.868,20.594,26.738,Subcase 1 (LC1),4.0
3,panel1,-66.730,20.466,36.481,Subcase 1 (LC1),4.0
4,panel2,-96.378,2.383,34.062,Subcase 1 (LC1),4.0
5,panel2,-88.315,25.010,26.685,Subcase 1 (LC1),4.0


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [7]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'max'
})
dataframeLength = len(panelLC1)
panelLC1['length'] = [length]*dataframeLength
panelLC1['width'] = [width]*dataframeLength
panelLC1['Load Case'] = ['LC1'] * dataframeLength


#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'max'
})
dataframeLength = len(panelLC2)
panelLC2['length'] = [length]*dataframeLength
panelLC2['width'] = [width]*dataframeLength
panelLC2['Load Case'] = ['LC2'] * dataframeLength


#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'max'
})
dataframeLength = len(panelLC3)
panelLC3['length'] = [length]*dataframeLength
panelLC3['width'] = [width]*dataframeLength
panelLC3['Load Case'] = ['LC3'] * dataframeLength


panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Load Case
Component Name,,,,,,,
panel1,-71.460333,17.88,29.705667,4.0,750,200,LC1
panel10,-71.460333,17.88,29.705667,4.0,750,200,LC1
panel2,-93.234667,17.88,29.610333,4.0,750,200,LC1
panel3,-93.370667,17.88,29.611667,4.0,750,200,LC1
panel4,-92.718000,17.88,30.064333,4.0,750,200,LC1
panel5,-92.224333,17.88,52.546333,4.0,750,200,LC1
panel6,-92.224333,17.88,52.546333,4.0,750,200,LC1
panel7,-92.718000,17.88,30.064333,4.0,750,200,LC1
panel8,-93.370667,17.88,29.611667,4.0,750,200,LC1


# Now we can apply the functions to the load cases 

In [8]:
panelLC1[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC2[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Load Case,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,,,,,
panel1,-71.460333,17.88,29.705667,4.0,750,200,LC1,5.624444,5.051195,1.019406
panel10,-71.460333,17.88,29.705667,4.0,750,200,LC1,5.624444,5.051195,1.019406
panel2,-93.234667,17.88,29.610333,4.0,750,200,LC1,5.624444,4.830948,0.769984
panel3,-93.370667,17.88,29.611667,4.0,750,200,LC1,5.624444,4.829521,0.768740
panel4,-92.718000,17.88,30.064333,4.0,750,200,LC1,5.624444,4.836411,0.772768
panel5,-92.224333,17.88,52.546333,4.0,750,200,LC1,5.624444,4.841700,0.661930
panel6,-92.224333,17.88,52.546333,4.0,750,200,LC1,5.624444,4.841700,0.661930
panel7,-92.718000,17.88,30.064333,4.0,750,200,LC1,5.624444,4.836411,0.772768
panel8,-93.370667,17.88,29.611667,4.0,750,200,LC1,5.624444,4.829521,0.768740


# Combining back into 1 Data Frame to output 

In [9]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=0)
outputDf = outputDf.reset_index()
outputDf

,Component Name,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Load Case,k_shear,k_biaxial,Reserve Factor
0,panel1,-71.460333,17.880000,29.705667,4.0,750,200,LC1,5.624444,5.051195,1.019406
1,panel10,-71.460333,17.880000,29.705667,4.0,750,200,LC1,5.624444,5.051195,1.019406
2,panel2,-93.234667,17.880000,29.610333,4.0,750,200,LC1,5.624444,4.830948,0.769984
3,panel3,-93.370667,17.880000,29.611667,4.0,750,200,LC1,5.624444,4.829521,0.768740
4,panel4,-92.718000,17.880000,30.064333,4.0,750,200,LC1,5.624444,4.836411,0.772768
5,panel5,-92.224333,17.880000,52.546333,4.0,750,200,LC1,5.624444,4.841700,0.661930
6,panel6,-92.224333,17.880000,52.546333,4.0,750,200,LC1,5.624444,4.841700,0.661930
7,panel7,-92.718000,17.880000,30.064333,4.0,750,200,LC1,5.624444,4.836411,0.772768
8,panel8,-93.370667,17.880000,29.611667,4.0,750,200,LC1,5.624444,4.829521,0.768740
9,panel9,-93.234667,17.880000,29.610333,4.0,750,200,LC1,5.624444,4.830948,0.769984


# Now we have to apply our reverse engineering and compute the thicknesses 

In [13]:
outputDf["thickness"] = outputDf.apply(panelBuckReverse,EModulus=EModulus, nu=nu, axis=1)
outputDf

,Component Name,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Load Case,k_shear,k_biaxial,Reserve Factor
0,panel1,-71.46,17.88,29.71,2.897621,750,200,LC1,5.62,5.05,1.02
1,panel10,-71.46,17.88,29.71,2.897621,750,200,LC1,5.62,5.05,1.02
2,panel2,-93.23,17.88,29.61,3.384232,750,200,LC1,5.62,4.83,0.77
3,panel3,-93.37,17.88,29.61,3.386772,750,200,LC1,5.62,4.83,0.77
4,panel4,-92.72,17.88,30.06,3.371474,750,200,LC1,5.62,4.84,0.77
5,panel5,-92.22,17.88,52.55,3.362372,750,200,LC1,5.62,4.84,0.66
6,panel6,-92.22,17.88,52.55,3.362372,750,200,LC1,5.62,4.84,0.66
7,panel7,-92.72,17.88,30.06,3.371474,750,200,LC1,5.62,4.84,0.77
8,panel8,-93.37,17.88,29.61,3.386772,750,200,LC1,5.62,4.83,0.77
9,panel9,-93.23,17.88,29.61,3.384232,750,200,LC1,5.62,4.83,0.77


In [14]:
#Drop irrelevant columns
outputDf = outputDf.drop(['length', 'width'], axis=1)
outputDf.head(3)


,Component Name,sigmaXX,sigmaYY,sigmaXY,thickness,Load Case,k_shear,k_biaxial,Reserve Factor
0,panel1,-71.46,17.88,29.71,2.897621,LC1,5.62,5.05,1.02
1,panel10,-71.46,17.88,29.71,2.897621,LC1,5.62,5.05,1.02
2,panel2,-93.23,17.88,29.61,3.384232,LC1,5.62,4.83,0.77


# ROUND & Output the file 

In [15]:
outputDf = outputDf.round(rounding_digits)
outputDf.to_csv(f'../data/{name}/output/processed_e.csv')
outputDf.head(10)

,Component Name,sigmaXX,sigmaYY,sigmaXY,thickness,Load Case,k_shear,k_biaxial,Reserve Factor
0,panel1,-71.46,17.88,29.71,2.90,LC1,5.62,5.05,1.02
1,panel10,-71.46,17.88,29.71,2.90,LC1,5.62,5.05,1.02
2,panel2,-93.23,17.88,29.61,3.38,LC1,5.62,4.83,0.77
3,panel3,-93.37,17.88,29.61,3.39,LC1,5.62,4.83,0.77
4,panel4,-92.72,17.88,30.06,3.37,LC1,5.62,4.84,0.77
5,panel5,-92.22,17.88,52.55,3.36,LC1,5.62,4.84,0.66
6,panel6,-92.22,17.88,52.55,3.36,LC1,5.62,4.84,0.66
7,panel7,-92.72,17.88,30.06,3.37,LC1,5.62,4.84,0.77
8,panel8,-93.37,17.88,29.61,3.39,LC1,5.62,4.83,0.77
9,panel9,-93.23,17.88,29.61,3.38,LC1,5.62,4.83,0.77
